In [ ]:
pip install tensorflow

In [ ]:
pip install nltk

In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import os
import nltk
import re
from nltk.stem import ISRIStemmer
nltk.download('punkt')
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv("/content/test _no_label.csv")
train = pd.read_excel("/content/train.xlsx")


train.columns = ['text','sentiment']
train_text = train['text']

# print(train)
# print(test)

In [ ]:
stemmer = ISRIStemmer()
def solve(word_list):
    arabic_reg = re.compile('[\u0600-\u06FF]+')
    arabic_words = [word for word in word_list if arabic_reg.fullmatch(word)]
    stemmed_words = [stemmer.stem(word) for word in arabic_words]
    return stemmed_words

def remove_and_stemm(tokenized_corpus):
    filtered_corpus = []
    for tokenized_document in tokenized_corpus:
        filtered_document = solve(tokenized_document)
        filtered_corpus.append(filtered_document)
    return filtered_corpus

In [ ]:
tokenized_train = [word_tokenize(sentence) for sentence in train_text]

stop_words_arabic = set(stopwords.words('arabic'))
filtered_texttrain = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_train
]

resulttrain=remove_and_stemm(filtered_texttrain)

cv=CountVectorizer()
todoc = [" ".join(doc) for doc in resulttrain]
x=cv.fit_transform(todoc)
feature_names = cv.get_feature_names_out()
xarr= x.toarray()
df = pd.DataFrame(xarr, columns=feature_names)

dftrain = pd.DataFrame({
    'review_description': resulttrain,
    'rating':train['sentiment']
})


tokenized_test = [word_tokenize(sentence) for sentence in test['review_description']]

filtered_texttest = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_test
]

resulttest=remove_and_stemm(filtered_texttest)
dftest = pd.DataFrame({
    'ID': test['ID'],
    'review_description': resulttest
})


final=[]
for i in resulttest:
  sentcount=[]
  for j in feature_names:
    count = i.count(j)
    sentcount.append(count)
  final.append(sentcount)


dftestcount = pd.DataFrame({
    'review_description':final
})
